In [19]:
import pandas as pd;\
db_orcal = pd.read_csv('db_oracle_11g.csv');\
container = pd.read_csv('dcos_container.csv')
docker = pd.read_csv('dcos_docker.csv')
mw_redis = pd.read_csv('mw_redis.csv')
os = pd.read_csv('os_linux.csv')

In [20]:
cmdb_group = db_orcal.groupby("cmdb_id");\
db = cmdb_group.get_group('db_003');\
db["time"] = pd.to_datetime(db.timestamp, unit='ms', origin='1970-01-01 08:00:00');\
db = db.sort_values(by='timestamp');\
metrics = db.groupby('name');\

/Users/ruyuexin/opt/anaconda3/envs/tf1/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
db_values = db[['name', 'timestamp', 'value']];\
db_values_1 = db_values.loc[db_values.timestamp<=1590077748000];\
db_values_1;\

,name,timestamp,value
0,CPU_free_pct,1590076800000,98.850566
21,CPU_Used_Pct,1590076813000,1.149434
39,Used_Tbs_Size,1590076826000,17450.400000
41,SEQ_Used_Pct,1590076826000,1.000000
75,LFParaWrite_Per_Sec,1590076834000,1.470000
...,...,...,...
7198,Call_Per_Sec,1590077737000,200.200000
7174,On_Off_State,1590077737000,1.000000
7168,Physical_Read_Per_Sec,1590077737000,1834.700000
7223,Login_Per_Sec,1590077737000,1.200000


In [37]:
from tsfresh import extract_features;\
ext_features = extract_features(db_values, column_id='name', column_sort='timestamp');\
#ext_features_t = ext_features.T
ext_features = ext_features.dropna(axis=1, how='any');\
ext_features = ext_features.to_numpy();\
ext_features.shape;\

Feature Extraction: 100%|██████████| 16/16 [00:02<00:00,  7.26it/s]


(46, 308)

In [38]:
from sklearn.decomposition import PCA;\
pca = PCA(n_components=5);\
ld_features = pca.fit_transform(ext_features);\
ld_features_t = ld_features.T;\
print(ld_features_t.shape);\
ld_features_t;\

(2, 46)


array([[-1.06397368e+12, -1.06397368e+12, -1.06397368e+12,
        -1.06397368e+12, -1.06397368e+12, -1.06397317e+12,
        -1.06396789e+12, -1.06397368e+12, -1.06396304e+12,
        -1.06397307e+12, -1.06397368e+12, -1.06397368e+12,
        -1.06397368e+12,  4.14970160e+13, -1.06397368e+12,
        -1.06394538e+12, -1.06396985e+12, -1.06397360e+12,
        -1.06397362e+12, -1.06397223e+12, -1.06397362e+12,
        -1.06397368e+12, -1.06397368e+12, -1.06397362e+12,
        -1.06286783e+12, -1.06397368e+12, -1.06397368e+12,
        -7.27156954e+11, -1.06397368e+12, -1.06397368e+12,
        -1.06397368e+12, -1.06397368e+12, -1.06397368e+12,
        -1.06396177e+12, -1.06397368e+12, -1.06397368e+12,
        -1.06397368e+12, -1.06397366e+12, -1.06397266e+12,
        -1.06397320e+12, -1.06397368e+12,  2.33466068e+12,
        -1.06397368e+12,  1.58120408e+12, -1.06397368e+12,
        -1.06397368e+12],
       [-4.42670740e+11, -4.42670740e+11, -4.42670740e+11,
        -4.42670740e+11, -4.42

In [39]:
import numpy as np
def is_invertible(a):
    print(np.linalg.matrix_rank(a))
    return a.shape[0] == a.shape[1] and np.linalg.matrix_rank(a) == a.shape[0]
a = np.random.rand(10, 10)
is_invertible(ld_features_t)

2


False

In [ ]:
from castle.algorithms import PC
#from castle.algorithms import DirectLiNGAM

import matplotlib.pyplot as plt
# structure learning
pc = PC();\
pc.learn(ld_features_t);\
adj_matrix = pc.causal_matrix

#g = DirectLiNGAM()
#g.learn(metrics)
#adj_matrix = g.causal_matrix

print(adj_matrix)